In [10]:
import pandas as pd
import numpy as np
from Bio import AlignIO
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF


In [36]:
# # Read entire file into a single string
# with open("data/alignments/BG_STRSQ.a2m", "r", encoding="utf-8") as f:
#     content = f.read()
#     content = list(set(list(content)))
#     content = list(set([x.lower() for x in content if (x.isalpha() or x=="-")]))
#     print(len(content))
#     print(content)
#     # print(set(output))

# dictionary = set()
# for sequence in sequences:
#     for elem in sequence:
#         dictionary.add(elem.upper())




In [39]:
print(len(dictionary))
print(dictionary)

25
{'-', 'E', 'H', 'P', 'M', '.', 'N', 'D', 'L', 'Y', 'A', 'I', 'Q', 'V', 'X', 'B', 'Z', 'F', 'C', 'R', 'G', 'K', 'W', 'S', 'T'}


In [41]:
a2m_file = "data/alignments/BG_STRSQ.a2m"   # path to your A2M file
window_size = 5            # context window size
test_size = 0.2            # fraction for test set
alphabet = list("ACDEFGHIKLMNPQRSTVWY-")  # 20 amino acids + gap

alignment = AlignIO.read(a2m_file, "fasta")  # "a2m" behaves like fasta
sequences = [str(record.seq) for record in alignment]
print(f"Loaded {len(sequences)} sequences from {a2m_file}")

Loaded 28048 sequences from data/alignments/BG_STRSQ.a2m


In [29]:
# Build context-target pairs

X_raw, y_raw = [], []
for seq in sequences:
    for i in range(len(seq) - window_size):
        context = list(seq[i : i + window_size])
        target = seq[i + window_size]
        if any(c not in alphabet for c in context + [target]):
            continue
        X_raw.append(context)
        y_raw.append(target)

print(f"Built {len(X_raw)} training examples (window={window_size})")


Built 11665223 training examples (window=5)


In [30]:
enc = OneHotEncoder(sparse_output=False, categories=[alphabet] * window_size)
X = enc.fit_transform(np.array(X_raw))

label_to_int = {c: i for i, c in enumerate(alphabet)}
y = np.array([label_to_int[c] for c in y_raw])

In [31]:
X_train, X_test, y_train, y_test, X_raw_train, X_raw_test = train_test_split(
    X, y, X_raw, test_size=test_size, random_state=42, stratify=y
)

In [40]:
kernel = 1.0 * RBF(length_scale=2.0)
gpc = GaussianProcessClassifier(kernel=kernel, random_state=42, max_iter_predict=200)
print("Fitting Gaussian Process Classifier...")
gpc.fit(X_train, y_train)

Fitting Gaussian Process Classifier...


MemoryError: Unable to allocate 634. TiB for an array with shape (9332178, 9332178) and data type float64

In [ ]:
probs_train = gpc.predict_proba(X_train)
probs_test = gpc.predict_proba(X_test)


In [ ]:
print("\n--- Training predictions (first 5) ---")
for context, p in zip(X_raw_train[:5], probs_train[:5]):
    dist = {cat: round(prob, 3) for cat, prob in zip(alphabet, p)}
    print(f"Context={context} → {dist}")

print("\n--- Test predictions (first 5) ---")
for context, p in zip(X_raw_test[:5], probs_test[:5]):
    dist = {cat: round(prob, 3) for cat, prob in zip(alphabet, p)}
    print(f"Context={context} → {dist}")